In [10]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image, UnidentifiedImageError
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def is_image_file(filename):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    return any(filename.lower().endswith(ext) for ext in valid_extensions)

def load_image(image_path):
    image = Image.open(image_path)
    image = image.resize((160, 160))
    image = np.asarray(image)
    image = image.astype('float32')
    mean, std = image.mean(), image.std()
    image = (image - mean) / std
    image = np.expand_dims(image, axis=0)
    return image

def load_pb_model(model_filepath):
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()
    with open(model_filepath, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def, name="")
    return graph

def generate_embeddings(graph, dataset_path):
    embeddings = []
    labels = []
    with tf.compat.v1.Session(graph=graph) as sess:
        for person_name in os.listdir(dataset_path):
            person_path = os.path.join(dataset_path, person_name)
            if os.path.isdir(person_path):
                for image_name in os.listdir(person_path):
                    if not is_image_file(image_name):
                        continue  # Skip files that are not images
                    image_path = os.path.join(person_path, image_name)
                    try:
                        image = load_image(image_path)
                    except UnidentifiedImageError:
                        print(f"Skipping file (not an image): {image_path}")
                        continue  # Skip files that cannot be identified as images
                    # Fetch the tensors
                    images_placeholder = graph.get_tensor_by_name("input:0")
                    embeddings_tensor = graph.get_tensor_by_name("embeddings:0")
                    phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")
                    feed_dict = {images_placeholder: image, phase_train_placeholder: False}
                    embedding = sess.run(embeddings_tensor, feed_dict=feed_dict)
                    embeddings.append(embedding)
                    labels.append(person_name)
    embeddings = np.asarray(embeddings).reshape(len(embeddings), -1)
    labels = np.asarray(labels)
    return embeddings, labels

model_filepath = '20180402-114759/20180402-114759.pb'
graph = load_pb_model(model_filepath)
dataset_path = 'dataset'
embeddings, labels = generate_embeddings(graph, dataset_path)


In [3]:
# import tensorflow as tf
# import numpy as np
# import os
# from PIL import Image
# from tqdm import tqdm

# def preprocess_image(image_path, target_size=(160, 160)):
#     """Load an image file, resize it to target size, and normalize it."""
#     image = Image.open(image_path)
#     image = image.resize(target_size)
#     image_array = np.array(image)
#     # Normalize the image
#     image_array = image_array.astype('float32')
#     mean, std = image_array.mean(), image_array.std()
#     image_array = (image_array - mean) / std
#     return image_array

# def load_dataset(dataset_path, target_size=(160, 160)):
#     """Load and preprocess the entire dataset."""
#     X, y = [], []
#     # Walk through the person directories and list their images
#     for person_name in tqdm(os.listdir(dataset_path), desc="Processing"):
#         person_path = os.path.join(dataset_path, person_name)
#         if not os.path.isdir(person_path):
#             continue  # Skip non-directory files
#         for image_name in os.listdir(person_path):
#             image_path = os.path.join(person_path, image_name)
#             if not os.path.isfile(image_path) or not image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
#                 continue  # Skip non-image files
#             image_array = preprocess_image(image_path, target_size)
#             X.append(image_array)
#             y.append(person_name)
#     return np.array(X), np.array(y)

# # Path to your dataset
# dataset_path = 'database'

# # Load and preprocess the dataset
# X, y = load_dataset(dataset_path)

# print(f"Loaded and preprocessed {len(X)} images.")


Processing: 100%|██████████| 4/4 [00:00<00:00, 23.43it/s]

Loaded and preprocessed 6 images.


In [11]:
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import joblib

# Assuming `embeddings` and `labels` are available from previous steps
# Encode the labels into integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Save the fitted LabelEncoder to a file
joblib.dump(label_encoder, 'label_encoder.joblib')

# Splitting the dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_encoded, test_size=0.2, random_state=42)

# Creating the classifier model
def build_classifier_model(input_shape, num_classes):
    inputs = Input(shape=(input_shape,))
    x = Dense(128, activation='relu')(inputs)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

# Building the model
num_classes = len(np.unique(labels_encoded))
classifier_model = build_classifier_model(X_train.shape[1], num_classes)

# Training the model
history = classifier_model.fit(X_train, y_train, 
                               epochs=10, 
                               validation_data=(X_test, y_test), 
                               batch_size=32)

Epoch 1/10
332/332 [==============================] - 2s 6ms/step - loss: 7.9495 - accuracy: 0.0499 - val_loss: 7.6188 - val_accuracy: 0.0640
Epoch 2/10
332/332 [==============================] - 2s 5ms/step - loss: 7.0336 - accuracy: 0.0881 - val_loss: 7.5065 - val_accuracy: 0.1126
Epoch 3/10
332/332 [==============================] - 2s 5ms/step - loss: 6.5549 - accuracy: 0.1227 - val_loss: 7.4611 - val_accuracy: 0.1434
Epoch 4/10
332/332 [==============================] - 1s 4ms/step - loss: 6.1129 - accuracy: 0.1536 - val_loss: 7.4626 - val_accuracy: 0.1755
Epoch 5/10
332/332 [==============================] - 2s 5ms/step - loss: 5.6555 - accuracy: 0.1850 - val_loss: 7.4498 - val_accuracy: 0.2044
Epoch 6/10
332/332 [==============================] - 2s 5ms/step - loss: 5.1327 - accuracy: 0.2240 - val_loss: 7.4214 - val_accuracy: 0.2361
Epoch 7/10
332/332 [==============================] - 2s 5ms/step - loss: 4.5943 - accuracy: 0.2670 - val_loss: 7.4284 - val_accuracy: 0.2609
Epoch 

In [12]:
# Evaluating the model
evaluation = classifier_model.evaluate(X_test, y_test)
print(f'Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}')

# Save the model and the label encoder for later use
classifier_model.save('face_recognition_classifier.h5')
# Ensure you also save `label_encoder` using joblib or pickle to decode predicted labels

83/83 [==============================] - 0s 3ms/step - loss: 7.5860 - accuracy: 0.3129
Test Loss: 7.5860443115234375, Test Accuracy: 0.31287649273872375


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import joblib  # For loading the label encoder

# Function to load the FaceNet model
def load_pb_model(model_filepath):
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()
    with open(model_filepath, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def, name="")
    return graph

# Load your trained classifier model and label encoder
classifier_model = load_model('face_recognition_classifier.h5')
label_encoder = joblib.load('label_encoder.joblib')

# Load the FaceNet model
facenet_model = load_pb_model('20180402-114759/20180402-114759.pb')


def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((160, 160))  # Resize as per FaceNet's requirements
    image = np.asarray(image)
    image = image.astype('float32')
    mean, std = image.mean(), image.std()
    image = (image - mean) / std
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image


def get_embedding(model, image):
    with tf.compat.v1.Session(graph=model) as sess:
        images_placeholder = model.get_tensor_by_name("input:0")
        embeddings_tensor = model.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = model.get_tensor_by_name("phase_train:0")
        feed_dict = {images_placeholder: image, phase_train_placeholder: False}
        embedding = sess.run(embeddings_tensor, feed_dict=feed_dict)
    return embedding


def predict_identity(image_path, confidence_threshold):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)
    
    # Generate embedding
    embedding = get_embedding(facenet_model, preprocessed_image)
    
    # Predict using the classifier model and get confidence scores
    prediction_scores = classifier_model.predict(embedding)
    confidence = np.max(prediction_scores)
    predicted_label_idx = np.argmax(prediction_scores, axis=1)

    print(confidence)
    
    if confidence < confidence_threshold:
        print("This person does not exist in the database.")
        return None
    else:
        predicted_label = label_encoder.inverse_transform(predicted_label_idx)
        print(f"Predicted person: {predicted_label[0]} with confidence: {confidence}")
        return predicted_label[0]

# Example usage
image_path = 'test/person/Kong_Quan_0001.jpg'  # Replace with your image path
predicted_person = predict_identity(image_path, confidence_threshold=0.1)  # Adjust threshold as needed

if predicted_person:
    print(f"Attendance marked for: {predicted_person}")
else:
    print("Unable to mark attendance, person not recognized.")


1/1 [==============================] - 0s 32ms/step
0.15961063
Predicted person: Hugo_Chavez with confidence: 0.15961062908172607
Attendance marked for: Hugo_Chavez
